# HOTRG (cytnx version)

* Author: Pochung Chen
* Last update: 2022/5/16
* Α α, Β β, Γ γ, Δ δ, Ε ε, Ζ ζ, Η η, Θ θ, Ι ι, Κ κ, Λ λ, Μ μ, Ν ν, Ξ ξ, Ο ο, Π π, Ρ ρ, Σ σ/ς, Τ τ, Υ υ, Φ φ, Χ χ, Ψ ψ, and Ω ω

Two site Hamiltonian: 
$$ \large
  E(s_1, s_2) = -Js_1 s_2
$$

Boltzmann weight on the bond:
$$ \large
  W = e^{-\beta E} = 
  \left[ \begin{array}{cc}
  e^{+\beta J} & e^{-\beta J} \\
  e^{-\beta J} & e^{+\beta J}
  \end{array} \right]
$$

Decompose $W$ as $M M^\dagger$, where
$$ \large
  W = e^{-\beta E} = M M^\dagger,
  M = 
  \left[ \begin{array}{cc}
  +\sqrt{\cosh{\beta J}} & +\sqrt{\sinh{\beta J}} \\
  +\sqrt{\cosh{\beta J}} & -\sqrt{\sinh{\beta J}}
  \end{array} \right]
$$

Numerically, we first perform SVD on $W$ to obtain $M$ and $M^\dagger$:
$$ \large
  W = U S V^\dagger \rightarrow
  M = U \sqrt{S}, M^\dagger = \sqrt{S} U^\dagger.
$$
Note that because $W$ is Hermitian, on has $V=U$.

$$ \large
  T_{ijkl} = \sum_\alpha
  M_{\alpha, i} M_{\alpha, j} M_{\alpha, k} M_{\alpha, l}
$$

## Cytnx cheat sheet

* numpy <-> cytnx.tensor <-> cytnx.UniTensor

In [1]:
import numpy as np
import cytnx

In [2]:
def ut_print(ut):
    ut.print_diagram()
    print(ut.get_block().numpy())

## Constrauct $M$

In [3]:
def W0_(T):
    W0 = np.array([[np.exp(+1/T),np.exp(-1/T)],
                   [np.exp(-1/T),np.exp(+1/T)]])
    return W0
W0_(1.0)

array([[2.71828183, 0.36787944],
       [0.36787944, 2.71828183]])

In [4]:
u, s, vd = np.linalg.svd(W0_(1.0))
print('s=\n', s)

u @ np.diag(np.sqrt(s))

s=
 [3.08616127 2.35040239]


array([[-1.24220797, -1.08406697],
       [-1.24220797,  1.08406697]])

In [5]:
def M0_(T):
    M0 = np.array([[+np.sqrt(np.cosh(1/T)), +np.sqrt(np.sinh(1/T))],
                   [+np.sqrt(np.cosh(1/T)), -np.sqrt(np.sinh(1/T))]])
    return M0

a_M0 = M0_(1)
print('[M0]=\n', a_M0)
print('[M0][M0d]=\n', a_M0 @ a_M0.transpose())
print('[M0][M0d]-[W0]=\n', a_M0 @ a_M0.transpose()-W0_(1.0))

[M0]=
 [[ 1.24220797  1.08406697]
 [ 1.24220797 -1.08406697]]
[M0][M0d]=
 [[2.71828183 0.36787944]
 [0.36787944 2.71828183]]
[M0][M0d]-[W0]=
 [[ 0.00000000e+00 -3.88578059e-16]
 [-3.88578059e-16  0.00000000e+00]]


In [6]:
def M_(T,h):
    W = np.array([[np.exp(1/T), np.exp(-1/T)],
                  [np.exp(-1/T), np.exp(1/T)]])
    W = cytnx.from_numpy(W)
    # S, U, Vd = cytnx.linalg.Svd(W)
    S, U, Vd = W.Svd()
    M = U @ cytnx.linalg.Diag(S.Pow(0.5))
    Md = cytnx.linalg.Diag(S.Pow(0.5)) @ Vd
    M = cytnx.UniTensor(M, rowrank=1)
    M.set_name('M')
    Md = cytnx.UniTensor(Md, rowrank=1)
    Md.set_name('M†')    
    return M, Md

ut_M, ut_Md = M_(1,0)

print('ut_M=')
ut_print(ut_M)

print('ut_Md=')
ut_print(ut_Md)

ut_M=
-----------------------
tensor Name : M
tensor Rank : 2
block_form  : false
is_diag     : False
on device   : cytnx device: CPU
            -------------      
           /             \     
     0 ____| 2         2 |____ 1  
           \             /     
            -------------      
[[-1.24220797 -1.08406697]
 [-1.24220797  1.08406697]]
ut_Md=
-----------------------
tensor Name : M†
tensor Rank : 2
block_form  : false
is_diag     : False
on device   : cytnx device: CPU
            -------------      
           /             \     
     0 ____| 2         2 |____ 1  
           \             /     
            -------------      
[[-1.24220797 -1.24220797]
 [-1.08406697  1.08406697]]


In [7]:
ut_Md.set_labels((1,2))
ut_print(ut_Md)

-----------------------
tensor Name : M†
tensor Rank : 2
block_form  : false
is_diag     : False
on device   : cytnx device: CPU
            -------------      
           /             \     
     1 ____| 2         2 |____ 2  
           \             /     
            -------------      
[[-1.24220797 -1.24220797]
 [-1.08406697  1.08406697]]


In [8]:
print(W0_(1.0))
ut_print(cytnx.Contract(ut_M, ut_Md))

[[2.71828183 0.36787944]
 [0.36787944 2.71828183]]
-----------------------
tensor Name : 
tensor Rank : 2
block_form  : false
is_diag     : False
on device   : cytnx device: CPU
            -------------      
           /             \     
     0 ____| 2         2 |____ 2  
           \             /     
            -------------      
[[2.71828183 0.36787944]
 [0.36787944 2.71828183]]


## Construct $T_{bare}$

In [9]:
def delta_():
    delta = cytnx.zeros([2,2,2,2])
    delta[0,0,0,0]= 1.0
    delta[1,1,1,1]= 1.0
    delta = cytnx.UniTensor(delta, rowrank=2)
    delta.set_name('ẟ')
    return delta

ut_delta = delta_()
# print(ut_delta.get_block().numpy())
# ut_delta.print_diagram()
ut_print(ut_delta)

-----------------------
tensor Name : ẟ
tensor Rank : 4
block_form  : false
is_diag     : False
on device   : cytnx device: CPU
            -------------      
           /             \     
     0 ____| 2         2 |____ 2  
           |             |     
     1 ____| 2         2 |____ 3  
           \             /     
            -------------      
[[[[1. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]]


 [[[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 1.]]]]


In [10]:
def T_bare_(T, h):
    ut_M, ut_Md = M_(T, h)
    ut_delta = delta_()
    Ising_net = cytnx.Network('Networks/Ising_square.net')
    Ising_net.PutUniTensors(['delta', 'M0.d', 'M1.d', 'M2', 'M3'],
                            [ut_delta, ut_Md, ut_Md, ut_M, ut_M])
    T_bare = Ising_net.Launch(optimal=True)
    T_bare.set_name('T_bare')
    return T_bare

ut_T = T_bare_(1, 0)
# ut_T.print_diagram()
# ut_T.get_block().numpy()
ut_print(ut_T)

-----------------------
tensor Name : T_bare
tensor Rank : 4
block_form  : false
is_diag     : False
on device   : cytnx device: CPU
            -------------      
           /             \     
     0 ____| 2         2 |____ 2  
           |             |     
     1 ____| 2         2 |____ 3  
           \             /     
            -------------      
[[[[ 4.76219569e+00  2.32739908e-15]
   [ 2.32739908e-15  3.62686041e+00]]

  [[ 2.05157350e-15  3.62686041e+00]
   [ 3.62686041e+00 -3.01055929e-15]]]


 [[[ 2.16101918e-15  3.62686041e+00]
   [ 3.62686041e+00 -3.21683176e-15]]

  [[ 3.62686041e+00 -2.77274255e-15]
   [-2.77274255e-15  2.76219569e+00]]]]


In [11]:
a_M0 = M0_(1)
a_T = np.zeros((2,2,2,2))
for i in range(2):
    for j in range(2):
        for k in range(2):
            for l in range(2):
                a_T[i,j,k,l]=a_M0[0,i]*a_M0[0,j]*a_M0[0,k]*a_M0[0,l]+a_M0[1,i]*a_M0[1,j]*a_M0[1,k]*a_M0[1,l]
a_T 

array([[[[4.76219569, 0.        ],
         [0.        , 3.62686041]],

        [[0.        , 3.62686041],
         [3.62686041, 0.        ]]],


       [[[0.        , 3.62686041],
         [3.62686041, 0.        ]],

        [[3.62686041, 0.        ],
         [0.        , 2.76219569]]]])

In [12]:
ymerge_net = cytnx.Network('Networks/merge_two_y.net')
print(ymerge_net)

==== Network ====
[x] Tu : 0 1 ; 3 -1 
[x] Td : -1 2 ; 4 5 
TOUT : 0 1 2 ; 3 4 5 
ORDER : 



In [13]:
ymerge_net.PutUniTensors(['Tu', 'Td'], [ut_T, ut_T])
ut_Tupdn = ymerge_net.Launch(optimal=True)
ut_Tupdn.print_diagram()

-----------------------
tensor Name : 
tensor Rank : 6
block_form  : false
is_diag     : False
on device   : cytnx device: CPU
            -------------      
           /             \     
     0 ____| 2         2 |____ 3  
           |             |     
     1 ____| 2         2 |____ 4  
           |             |     
     2 ____| 2         2 |____ 5  
           \             /     
            -------------      


In [14]:
TM = ut_Tupdn.Trace(0, 5, by_label=True)
TM = TM.get_block()
TM.reshape_(4,4)
λ, U = cytnx.linalg.Eigh(TM)
λ.numpy()

array([ 0.94512664,  0.98168436, 53.59815003, 55.67133903])

In [15]:
E = -np.log(λ.numpy())
E

array([ 0.05643635,  0.01848545, -3.98151455, -4.01946545])